In [78]:
from pyspark import SparkContext, SparkConf

conf=SparkConf().setAppName("sample")

sc=SparkContext

In [79]:
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession


conf = SparkConf().setAppName("sample")

sc = SparkContext.getOrCreate(conf=conf)


spark= SparkSession.builder.appName("pract").getOrCreate()


df=spark.read.csv("dept.csv", header=True, inferSchema=True)

df.show()
df.printSchema()

+-------------+----------+
|accountNumber|assetValue|
+-------------+----------+
|        DEPT3|       500|
|        DEPT3|       300|
|        DEPT1|      1000|
|        DEPT1|       700|
|        DEPT1|       500|
|        DEPT2|       400|
|        DEPT2|       200|
+-------------+----------+

root
 |-- accountNumber: string (nullable = true)
 |-- assetValue: integer (nullable = true)



In [80]:
## now we have to tackel with the error regarding the infer schema


In [81]:
text_rdd=sc.textFile("Hello Shubham.txt")

print(text_rdd.collect())


['Hello Shubham', 'hello Vishnu ', 'Hello Charan', 'Hello Bharath']


In [82]:
rdd1=sc.parallelize([1,2,3,4,5,6])

rdd2=sc.parallelize([3,4,5,6,7,8,9])

## Union function==> it make one single rdd with contain all duplicates also

rdd3=rdd1.union(rdd2)

print(rdd3.collect())

[1, 2, 3, 4, 5, 6, 3, 4, 5, 6, 7, 8, 9]


In [83]:
rdd1=sc.parallelize([1,2,3,4,5,6])

rdd2=sc.parallelize([3,4,5,6,7,8,9])

## Intersection==> it gives ud only cpmmon values.

rdd3=rdd1.intersection(rdd2)

print(rdd3.collect())

[4, 5, 6, 3]


In [84]:
rdd1=sc.parallelize([(1, "Shubham"),(2, "Charan"),(3, "Bharath")])

rdd2=sc.parallelize([(1, "Vishnu"), (2, "Charan")])

rdd3=rdd1.join(rdd2)
### Join function==> this function give the common values on the basis of the Key.
print(rdd3.collect())

[(1, ('Shubham', 'Vishnu')), (2, ('Charan', 'Charan'))]


In [85]:
rdd1=sc.parallelize([(1, "Shubham"),(2, "Charan")])

rdd2=sc.parallelize([(1, "Vishnu"), (2, "Charan")])

In [86]:
from pyspark.sql import SparkSession

spark=SparkSession\
          .builder\
          .config("spark.jars.packages","org.apache.spark:spark-avro_2.13:4.0.1")\
          .appName("prac")\
          .getOrCreate()
sc=spark.sparkContext
## createDataFrame(Data,column[])

df=spark.createDataFrame(rdd1,['id','name'])

df.show() ## to get result of

df.printSchema() ##  show the schema

+---+-------+
| id|   name|
+---+-------+
|  1|Shubham|
|  2| Charan|
+---+-------+

root
 |-- id: long (nullable = true)
 |-- name: string (nullable = true)



In [87]:
## Aply our Schemas to our data set(manual way to apply schema)
from pyarrow import StructScalar

from pyspark.sql.types import StructType,StructField,IntegerType,StringType

schema=StructType([
    StructField("id",IntegerType(),True),## True means we accepting the null values
    StructField("name",StringType(),True)
])

rdd1=sc.parallelize([(1, "Shubham"),(2, "Charan")])

df=spark.createDataFrame(rdd1,schema = schema)

df.show()

df.printSchema()

+---+-------+
| id|   name|
+---+-------+
|  1|Shubham|
|  2| Charan|
+---+-------+

root
 |-- id: integer (nullable = true)
 |-- name: string (nullable = true)



In [88]:
schema=StructType([
    StructField("id",IntegerType(),False),## False means we not accepting the null values
    StructField("name",StringType(),False)
])

rdd1=sc.parallelize([(1, "Shubham"),(2, "Charan")])

df=spark.createDataFrame(rdd1,schema=schema)

df.show()

df.printSchema()

+---+-------+
| id|   name|
+---+-------+
|  1|Shubham|
|  2| Charan|
+---+-------+

root
 |-- id: integer (nullable = false)
 |-- name: string (nullable = false)



In [89]:
## how to read and load diff file format in spark
## 1. Cvs files

csv_df=spark.read.csv("dept.csv")

csv_df.show() ## here we get data from our csv but column name are missplaced we got _c(column)

csv_df.printSchema()

+-------------+----------+
|          _c0|       _c1|
+-------------+----------+
|accountNumber|assetValue|
|        DEPT3|       500|
|        DEPT3|       300|
|        DEPT1|      1000|
|        DEPT1|       700|
|        DEPT1|       500|
|        DEPT2|       400|
|        DEPT2|       200|
+-------------+----------+

root
 |-- _c0: string (nullable = true)
 |-- _c1: string (nullable = true)



In [90]:
csv_df=spark.read.csv("dept.csv",header=True) ## by using Header= True we get actul format with name of column

csv_df.show()

csv_df.printSchema() ## by default when we read a csv file the data type if default gets as a string type .

+-------------+----------+
|accountNumber|assetValue|
+-------------+----------+
|        DEPT3|       500|
|        DEPT3|       300|
|        DEPT1|      1000|
|        DEPT1|       700|
|        DEPT1|       500|
|        DEPT2|       400|
|        DEPT2|       200|
+-------------+----------+

root
 |-- accountNumber: string (nullable = true)
 |-- assetValue: string (nullable = true)



In [91]:
## for accesing actual data type we have to use InferSchema = True to automnatically get the actual data type of our data.
schema=StructType([
    StructField("accountNumber",StringType(),True),
    StructField("assetValue",IntegerType(),True)
])
csv_df=spark.read.csv("dept.csv",header=True, schema=schema)

csv_df.show()

csv_df.printSchema()

## Inferschema is usefull in only for Small data type in huge data it not work
## that why wem have to sppy schema separately to call .

+-------------+----------+
|accountNumber|assetValue|
+-------------+----------+
|        DEPT3|       500|
|        DEPT3|       300|
|        DEPT1|      1000|
|        DEPT1|       700|
|        DEPT1|       500|
|        DEPT2|       400|
|        DEPT2|       200|
+-------------+----------+

root
 |-- accountNumber: string (nullable = true)
 |-- assetValue: integer (nullable = true)



In [92]:
## 2. Perquet files
par_df=spark.read.parquet("data.parquet") ## just accesing the data it takes default data type as Strring


par_df.show()

par_df.printSchema()

+--------+----------+--------+----------------+
|      id|     tdate|category|         product|
+--------+----------+--------+----------------+
|00000000|06-26-2011|Exercise|Gymnastics Rings|
|00000002|06-01-2011|Exercise|Gymnastics Rings|
+--------+----------+--------+----------------+

root
 |-- id: string (nullable = true)
 |-- tdate: string (nullable = true)
 |-- category: string (nullable = true)
 |-- product: string (nullable = true)



In [93]:
## 3. Orc file

orc_df=spark.read.orc("data.orc")

orc_df.show()

orc_df.printSchema() ## default string type

+----------+---------+--------------------+------------------+-----------+----------+-----+-----+---+------------+------------+--------------------+--------------------+
|first_name|last_name|        company_name|           address|       city|    county|state|  zip|age|      phone1|      phone2|               email|                 web|
+----------+---------+--------------------+------------------+-----------+----------+-----+-----+---+------------+------------+--------------------+--------------------+
|   Solange|   Shinko|   Mosocco, Ronald A|       426 Wolf St|   Metairie| Jefferson|   LA|70002| 21|504-979-9175|504-265-8174|  solange@shinko.com|http://www.mosocc...|
|    Arlene|  Klusman|Beck Horizon Buil...|        3 Secor Rd|New Orleans|   Orleans|   LA|70112| 20|504-710-5840|504-946-1807|arlene_klusman@gm...|http://www.beckho...|
|     Larae|   Gudroe|Lehigh Furn Divsn...| 6651 Municipal Rd|      Houma|Terrebonne|   LA|70360| 33|985-890-7262|985-261-5783|larae_gudroe@gmai...|ht

In [94]:
### 4 avro file
from pyspark.sql import SparkSession

spark=SparkSession\
          .builder\
          .config("spark.jars.packages","org.apache.spark:spark-avro_2.11:4.0.0")\
          .appName("prac")\
          .getOrCreate()
avro_df=spark.read.format("avro").load("data.avro")

avro_df.show()

avro_df.printSchema()

## hre all sytax is correct but by colab we get error

## Maven Reposetory:to get sytaxt to call avro file


AnalysisException: Failed to find data source: avro. Avro is built-in but external data source module since Spark 2.4. Please deploy the application as per the deployment section of Apache Avro Data Source Guide.

In [95]:
## to get our spark version
print(spark.version)

4.0.1


In [105]:
### 5.json file
json_df=spark.read.json("pets1.json")

json_df.show()

json_df.printSchema()

+-------+--------+----+----------+
|Boolean|  Mobile|Name|      Pets|
+-------+--------+----+----------+
|   true|12345678|Test|[Dog, cat]|
+-------+--------+----+----------+

root
 |-- Boolean: boolean (nullable = true)
 |-- Mobile: long (nullable = true)
 |-- Name: string (nullable = true)
 |-- Pets: array (nullable = true)
 |    |-- element: string (containsNull = true)

